In [149]:
%matplotlib notebook
import cvxpy as cp
import dccp
import torch
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn.metrics import zero_one_loss, confusion_matrix
from scipy.io import arff
import pandas as pd
import time
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import matplotlib.patches as mpatches
import json
import random
import math
import os, psutil
from datetime import datetime
import torch.nn as nn
from torch.autograd import Variable

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
np.random.seed(0)

TRAIN_SLOPE = 1
EVAL_SLOPE = 5
X_LOWER_BOUND = -10
X_UPPER_BOUND = 10

In [150]:
def load_spam_data():
    torch.manual_seed(0)
    np.random.seed(0)
    path = r"C:\Users\sagil\Desktop\nir_project\tip_spam_data\IS_journal_tip_spam.arff"
    data, meta = arff.loadarff(path)
    df = pd.DataFrame(data)
    most_disc = ['qTips_plc', 'rating_plc', 'qEmail_tip', 'qContacts_tip', 'qURL_tip', 'qPhone_tip', 'qNumeriChar_tip', 'sentistrength_tip', 'combined_tip', 'qWords_tip', 'followers_followees_gph', 'qunigram_avg_tip', 'qTips_usr', 'indeg_gph', 'qCapitalChar_tip', 'class1']
    df = df[most_disc]
    df["class1"].replace({b'spam': -1, b'notspam': 1}, inplace=True)
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    Y = df['class1'].values
    X = df.drop('class1', axis = 1).values
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    return torch.from_numpy(X), torch.from_numpy(Y)

In [151]:
X, Y = load_spam_data()

In [152]:
lam = 1e-4

class CAE(nn.Module):
    def __init__(self):
        super(CAE, self).__init__()

        self.fc1 = nn.Linear(15, 12, bias = True) # Encoder
        self.fc2 = nn.Linear(12, 15, bias = True) # Decoder

        self.sigmoid = nn.Sigmoid()


    def encoder(self, x):
        h1 = self.sigmoid(self.fc1(x.view(-1, 15)))
        return h1

    def decoder(self,z):
        h2 = (self.fc2(z))
        return h2

    def forward(self, x):
            h1 = self.encoder(x)
            h2 = self.decoder(h1)
            return h1, h2

mse_loss = nn.MSELoss(size_average = False)

def loss_function(W, x, recons_x, h, lam):
    """Compute the Contractive AutoEncoder Loss
    Evalutes the CAE loss, which is composed as the summation of a Mean
    Squared Error and the weighted l2-norm of the Jacobian of the hidden
    units with respect to the inputs.
    See reference below for an in-depth discussion:
      #1: http://wiseodd.github.io/techblog/2016/12/05/contractive-autoencoder
    Args:
        `W` (FloatTensor): (N_hidden x N), where N_hidden and N are the
          dimensions of the hidden units and input respectively.
        `x` (Variable): the input to the network, with dims (N_batch x N)
        recons_x (Variable): the reconstruction of the input, with dims
          N_batch x N.
        `h` (Variable): the hidden units of the network, with dims
          batch_size x N_hidden
        `lam` (float): the weight given to the jacobian regulariser term
    Returns:
        Variable: the (scalar) CAE loss
    """
    mse = mse_loss(recons_x, x)
    # Since: W is shape of N_hidden x N. So, we do not need to transpose it as
    # opposed to #1
    dh = h * (1 - h) # Hadamard product produces size N_batch x N_hidden
    # Sum through the input dimension to improve efficiency, as suggested in #1
    w_sum = torch.sum(Variable(W)**2, dim=1)
    # unsqueeze to avoid issues with torch.mv
    w_sum = w_sum.unsqueeze(1) # shape N_hidden x 1
    contractive_loss = torch.sum(torch.mm(dh**2, w_sum), 0)
    return mse + contractive_loss.mul_(lam)

In [153]:
model = CAE()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
train_dset = TensorDataset(X, Y)
train_loader = DataLoader(train_dset, batch_size=64, shuffle=True)
print(len(X[0]))

def train(epoch):
    model.train()
    train_loss = 0

    

    for idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        optimizer.zero_grad()

        hidden_representation, recons_x = model(data)

        # Get the weights
        # model.state_dict().keys()
        # change the key by seeing the keys manually.
        # (In future I will try to make it automatic)
        W = model.state_dict()['fc1.weight']
        loss = loss_function(W, data.view(-1, 15), recons_x,
                             hidden_representation, lam)

        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()

        if idx % 12 == 0:
            print('Train epoch: {} [{}/{}({:.0f}%)]\t Loss: {:.6f}'.format(
                  epoch, idx*len(data), len(train_loader.dataset),
                  100*idx/len(train_loader),
                  loss.data[0]/len(data)))


    print('====> Epoch: {} Average loss: {:.4f}'.format(
         epoch, train_loss / len(train_loader.dataset)))

    
epochs = 50
for epoch in range(epochs):
    train(epoch)

15
Train epoch: 0 [0/7076(0%)]	 Loss: 17.007141
Train epoch: 0 [768/7076(11%)]	 Loss: 11.078425
Train epoch: 0 [1536/7076(22%)]	 Loss: 64.115066
Train epoch: 0 [2304/7076(32%)]	 Loss: 13.035456
Train epoch: 0 [3072/7076(43%)]	 Loss: 9.498035
Train epoch: 0 [3840/7076(54%)]	 Loss: 12.457956
Train epoch: 0 [4608/7076(65%)]	 Loss: 12.073766
Train epoch: 0 [5376/7076(76%)]	 Loss: 9.971311
Train epoch: 0 [6144/7076(86%)]	 Loss: 15.331076
Train epoch: 0 [6912/7076(97%)]	 Loss: 10.501161
====> Epoch: 0 Average loss: 14.7556
Train epoch: 1 [0/7076(0%)]	 Loss: 12.750724
Train epoch: 1 [768/7076(11%)]	 Loss: 9.905615
Train epoch: 1 [1536/7076(22%)]	 Loss: 18.047053
Train epoch: 1 [2304/7076(32%)]	 Loss: 11.176498
Train epoch: 1 [3072/7076(43%)]	 Loss: 9.846117
Train epoch: 1 [3840/7076(54%)]	 Loss: 9.559809
Train epoch: 1 [4608/7076(65%)]	 Loss: 10.406567
Train epoch: 1 [5376/7076(76%)]	 Loss: 13.613091
Train epoch: 1 [6144/7076(86%)]	 Loss: 16.666820
Train epoch: 1 [6912/7076(97%)]	 Loss: 9.563

Train epoch: 16 [1536/7076(22%)]	 Loss: 2.383086
Train epoch: 16 [2304/7076(32%)]	 Loss: 1.598564
Train epoch: 16 [3072/7076(43%)]	 Loss: 1.367749
Train epoch: 16 [3840/7076(54%)]	 Loss: 1.476131
Train epoch: 16 [4608/7076(65%)]	 Loss: 2.640110
Train epoch: 16 [5376/7076(76%)]	 Loss: 1.533226
Train epoch: 16 [6144/7076(86%)]	 Loss: 3.977267
Train epoch: 16 [6912/7076(97%)]	 Loss: 2.059634
====> Epoch: 16 Average loss: 3.8687
Train epoch: 17 [0/7076(0%)]	 Loss: 3.997765
Train epoch: 17 [768/7076(11%)]	 Loss: 3.449029
Train epoch: 17 [1536/7076(22%)]	 Loss: 1.164876
Train epoch: 17 [2304/7076(32%)]	 Loss: 1.004632
Train epoch: 17 [3072/7076(43%)]	 Loss: 2.389946
Train epoch: 17 [3840/7076(54%)]	 Loss: 2.070458
Train epoch: 17 [4608/7076(65%)]	 Loss: 1.205492
Train epoch: 17 [5376/7076(76%)]	 Loss: 2.223591
Train epoch: 17 [6144/7076(86%)]	 Loss: 1.294128
Train epoch: 17 [6912/7076(97%)]	 Loss: 1.377809
====> Epoch: 17 Average loss: 3.7008
Train epoch: 18 [0/7076(0%)]	 Loss: 3.665941
Trai

Train epoch: 32 [2304/7076(32%)]	 Loss: 1.231655
Train epoch: 32 [3072/7076(43%)]	 Loss: 2.480570
Train epoch: 32 [3840/7076(54%)]	 Loss: 0.996914
Train epoch: 32 [4608/7076(65%)]	 Loss: 1.407847
Train epoch: 32 [5376/7076(76%)]	 Loss: 1.650281
Train epoch: 32 [6144/7076(86%)]	 Loss: 1.720076
Train epoch: 32 [6912/7076(97%)]	 Loss: 1.056722
====> Epoch: 32 Average loss: 2.3727
Train epoch: 33 [0/7076(0%)]	 Loss: 0.719012
Train epoch: 33 [768/7076(11%)]	 Loss: 1.985002
Train epoch: 33 [1536/7076(22%)]	 Loss: 1.034068
Train epoch: 33 [2304/7076(32%)]	 Loss: 1.167551
Train epoch: 33 [3072/7076(43%)]	 Loss: 0.871024
Train epoch: 33 [3840/7076(54%)]	 Loss: 1.291612
Train epoch: 33 [4608/7076(65%)]	 Loss: 2.232327
Train epoch: 33 [5376/7076(76%)]	 Loss: 1.332566
Train epoch: 33 [6144/7076(86%)]	 Loss: 0.955890
Train epoch: 33 [6912/7076(97%)]	 Loss: 1.790083
====> Epoch: 33 Average loss: 2.3111
Train epoch: 34 [0/7076(0%)]	 Loss: 1.029960
Train epoch: 34 [768/7076(11%)]	 Loss: 1.408693
Train

Train epoch: 48 [768/7076(11%)]	 Loss: 1.949248
Train epoch: 48 [1536/7076(22%)]	 Loss: 1.198720
Train epoch: 48 [2304/7076(32%)]	 Loss: 1.274551
Train epoch: 48 [3072/7076(43%)]	 Loss: 2.975221
Train epoch: 48 [3840/7076(54%)]	 Loss: 1.248746
Train epoch: 48 [4608/7076(65%)]	 Loss: 0.647761
Train epoch: 48 [5376/7076(76%)]	 Loss: 1.526706
Train epoch: 48 [6144/7076(86%)]	 Loss: 1.439393
Train epoch: 48 [6912/7076(97%)]	 Loss: 2.212897
====> Epoch: 48 Average loss: 1.7536
Train epoch: 49 [0/7076(0%)]	 Loss: 0.905838
Train epoch: 49 [768/7076(11%)]	 Loss: 1.543641
Train epoch: 49 [1536/7076(22%)]	 Loss: 1.479976
Train epoch: 49 [2304/7076(32%)]	 Loss: 1.577976
Train epoch: 49 [3072/7076(43%)]	 Loss: 1.019518
Train epoch: 49 [3840/7076(54%)]	 Loss: 0.458398
Train epoch: 49 [4608/7076(65%)]	 Loss: 0.862520
Train epoch: 49 [5376/7076(76%)]	 Loss: 0.541237
Train epoch: 49 [6144/7076(86%)]	 Loss: 0.916780
Train epoch: 49 [6912/7076(97%)]	 Loss: 0.731394
====> Epoch: 49 Average loss: 1.7240


In [154]:
x = X[:1]
x_hidden, x_recons = model(x)
print(x)
print(x_recons)
print(x_hidden)

tensor([[-0.4247, -1.5674, -0.3062, -0.3391, -0.1670, -0.2742, -0.3558, -0.4824,
          0.2893, -0.0022, -0.2651,  1.1561, -0.2323, -0.1833, -0.1973]])
tensor([[-0.3345, -1.4632, -0.3104, -0.2819, -0.0866, -0.1944, -0.1665, -0.3003,
          0.0552,  0.2045,  0.0570,  1.0599, -0.3873, -0.3635, -0.3822]],
       grad_fn=<AddmmBackward>)
tensor([[0.3667, 0.2293, 0.4659, 0.4175, 0.7066, 0.8994, 0.5776, 0.4499, 0.4000,
         0.8964, 0.6821, 0.3903]], grad_fn=<SigmoidBackward>)


In [256]:
def func(X):
    return model(X)[0]

In [259]:
J = torch.autograd.functional.jacobian(func, X[0])[0]

In [260]:
print(J.size())
# print(torch.transpose(J, 1, 2))

torch.Size([12, 15])


In [275]:
U, S, V = torch.svd(J.T)

In [276]:
print(U.size())
print(S.size())
print(V.size())
print(S)

torch.Size([15, 12])
torch.Size([12])
torch.Size([12, 12])
tensor([0.3768, 0.3099, 0.2881, 0.2663, 0.2573, 0.2476, 0.2388, 0.2152, 0.2019,
        0.1639, 0.0953, 0.0841])


In [205]:
eps = 0.1
print(S>eps)
Bx = U[:, S>eps]

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
        False, False])


In [207]:
print(U.size())
print(Bx.size())

torch.Size([15, 12])
torch.Size([15, 10])


In [223]:
class DELTA():
    def __init__(self, x_dim):
        
        self.x = cp.Variable(x_dim)
        self.v = cp.Variable(1)
        self.r = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.w = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.B = cp.Parameter((x_dim, 1), value = np.random.randn(x_dim, 1))

        target = self.x@self.w - cp.sum_squares(self.x - self.r)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND, self.B@self.v == self.x-self.r]
        objective = cp.Maximize(target)
        problem = cp.Problem(objective, constraints)
        self.layer = CvxpyLayer(problem, parameters=[self.r, self.w, self.B],
                                variables=[self.x, self.v])
        
    def optimize_x(self, x, w, B):
        return self.layer(x, w, B)

x_dim = 3
delta = DELTA(x_dim)

B = torch.Tensor([[1],[2],[1]])
w = torch.ones(x_dim)
x = torch.ones(x_dim)
x_opt = delta.optimize_x(x, w, B)
print(x_opt)

(tensor([1.3333, 1.6667, 1.3333]), tensor([0.3333]))
